In [28]:
import carla
import time
from intersection_scenario_manager import IntersectionScenario
import numpy as np
import sys
sys.path.append("cartesian_to_frenet/")
from topology_extraction import get_opendrive_tree, get_junction_topology, get_junction_roads_topology
sys.path.append("../../global_route_planner/")
from global_planner import get_client, draw_waypoints, spawn_vehicle

In [29]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

all_waypoints = world.get_map().generate_waypoints(1)
def draw(road_id, lane_id=None, color=[0,255,0]):
    for wp in all_waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            draw_waypoints(world, [wp], 10, color) 
#             print(wp.lane_id)
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            draw_waypoints(world, [wp], 10, color) 
            
tree = get_opendrive_tree(world)
junction_topology = get_junction_topology(tree)
road_topology = get_junction_roads_topology(tree)

Connection to CARLA server established!


In [30]:
SM = IntersectionScenario(client)

Intersection Manager Initialized...


dict_keys([224, 965, 421, 1175, 905, 1162, 139, 1260, 685, 334, 751, 1148, 1050, 53, 599, 1070, 943, 509, 924, 829, 245])

In [31]:
junctionId = 53

In [45]:
ego_vehicle, my_vehicles, incoming_road_lane_id_to_outgoing_lane_id_dict, intersection_topology, ego_key, global_path_wps = SM.reset(num_vehicles=10, warm_start_duration=20, junction_id=junctionId)

generating scenario at junction id:  53
Control handed to system....


In [55]:
incoming_road_lane_id_to_outgoing_lane_id_dict

{(4, -2): [(5, -2, 63, (-2,)), (8, -2, 92, (-1, -2))],
 (4, -1): [(5, -1, 63, (-1,)), (7, 1, 80, (-1,))],
 (5, 1): [(8, -1, 104, (-1,)), (4, 1, 62, (1,))],
 (5, 2): [(4, 2, 62, (2,)), (7, 2, 100, (-1, -2))],
 (7, -2): [(8, -2, 126, (-2,)), (4, 2, 84, (2, 1))],
 (7, -1): [(5, -1, 96, (1,)), (8, -1, 126, (-1,))],
 (8, 1): [(7, 1, 125, (1,)), (4, 1, 88, (1,))],
 (8, 2): [(7, 2, 125, (2,)), (5, -2, 108, (2, 1))]}

In [71]:
intersecting_left, intersecting_right, parallel_same_dir, parallel_opposite_dir = intersection_topology


In [79]:
intersecting_left

[[(5, 2), (62, 2), (4, 2)], [(5, 1), (62, 1), (4, 1)]]

In [76]:
for element in intersecting_right:
    for item in element:
        draw(item[0], item[1])

In [46]:
ego_nearest_waypoint = client.get_world().get_map().get_waypoint(
    ego_vehicle.get_location(), project_to_road=True
)
client.get_world().debug.draw_string(
    ego_nearest_waypoint.transform.location,
    "O",
    draw_shadow=False,
    color=carla.Color(r=255, g=0, b=0),
    life_time=100,
)

In [44]:
ego_vehicle.destroy()
for actor in my_vehicles:
    actor.destroy()